In [ ]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import dmanip, vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# get data
data = pd.read_csv('/content/drive/My Drive/dataset/Level_1_NonPath_Path_GN_2286.csv') 

# split data
X=data.iloc[:,1:151]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

          Xc1.A      Xc1.R  ...  Grantham.Xd.29  Grantham.Xd.30
0     14.767060  10.829177  ...        0.032505        0.032749
1     11.728905   8.138424  ...        0.031626        0.032946
2      8.319776   2.773259  ...        0.031596        0.031098
3     10.255686   3.778411  ...        0.031188        0.031233
4      6.745884   2.698354  ...        0.031939        0.030060
...         ...        ...  ...             ...             ...
2281  12.828743   5.186088  ...        0.031884        0.033215
2282   6.040447   3.294790  ...        0.029505        0.027986
2283   3.997220   1.598888  ...        0.031122        0.031575
2284   3.739325   1.602568  ...        0.031110        0.031553
2285   9.393071   2.415361  ...        0.033175        0.032958

[2286 rows x 150 columns]
0       1
1       1
2       1
3       1
4       1
       ..
2281    2
2282    2
2283    2
2284    2
2285    2
Name: Class, Length: 2286, dtype: category
Categories (2, int64): [1, 2]


In [ ]:
# init
n_epochs = 10
n_comps = X.shape[1]

In [ ]:
# encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

#from collections import Counter
#from imblearn.over_sampling import SMOTE 


#sm = SMOTE(random_state=45)
#x_b, y_b = sm.fit_resample(X, yy)
#print('Resampled dataset shape %s' % Counter(Y_res))
# split the dataset
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25,
                                                    random_state=56,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

In [ ]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
#model.add(Dense(128, input_dim = 90, activation = 'relu'))
#model.add(Dense(64, activation = 'relu'))
model.add(Dense(1 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 150, 150)          91200     
_________________________________________________________________
lstm_13 (LSTM)               (None, 150, 128)          142848    
_________________________________________________________________
lstm_14 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 283,521
Trainable params: 283,521
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# compile model
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=64,
                    validation_data=(x_test, y_test))

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)

In [ ]:
test_pred

In [ ]:
from collections import Counter
#print(Counter(train_pred))
print(Counter(test_pred))

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))